# Import

In [1]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")' SwiftCoreMLTools
%install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")
		SwiftCoreMLTools
	.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")
		Just
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpk97hygt9/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/dduan/Just.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
Cloning https://github.com/dduan/Just.git
Resolving https://github.com/dduan/Just.git at 0.8.0
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.4
[1/3] Compiling Just Just.swift
[2/3] Compiling SwiftProtobuf AnyMessageStorage.swift
[3/4] Compiling SwiftCoreMLTools Activations.swift
[4/5] Compiling jupyterInstalledPackages jupyterInstalled

In [2]:
import Foundation
import TensorFlow
import SwiftCoreMLTools
import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// !wget -O "../data/housing.csv" https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data

if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
    try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
}

# Data Ingestion

In [4]:
// TODO: Download housing.csv

let data = try String(contentsOfFile:"../data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

let trainFeatures = Array(dataFeatures[0..<numTrainRecords])
let trainLabels = Array(dataLabels[0..<numTrainRecords])
let testFeatures = Array(dataFeatures[numTrainRecords...])
let testLabels = Array(dataLabels[numTrainRecords...])

print("Training shapes [\(trainFeatures.count) \(trainFeatures[0].count)] [\(trainLabels.count) \(trainLabels[0].count)]")
print("Testing shapes  [\(testFeatures.count) \(testFeatures[0].count)] [\(testLabels.count) \(testLabels[0].count)]")

let xAllTrain = Array(trainFeatures.joined())
let yAllTrain = Array(trainLabels.joined())
let xAllTest = Array(testFeatures.joined())
let yAllTest = Array(testLabels.joined())

let XTrainDeNorm = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([numTrainRecords, numColumns-1]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, 1]))
let XTestDeNorm = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numTestRecords, numColumns-1]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, 1]))

Training shapes [405 13] [405 1]
Testing shapes  [101 13] [101 1]


# Normalize

In [5]:
let mean = XTrainDeNorm.mean(alongAxes: 0)
let std = XTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[  2.0137098,   14.197531,    9.523555, 0.086419754,  0.53213036,   6.3311296,    64.47929,
    4.1678762,    6.832099,   353.68396,    18.03163,   379.84735,   11.394517]] [[ 6.5076075,  25.258776,   6.534038, 0.28098345, 0.11449408,  0.7311985,  29.000755,  2.1797554,
    6.544165,  132.14561,  2.2173452,  40.494495,   6.852825]]


In [6]:
let XTrain = (XTrainDeNorm - mean)/std
let XTest = (XTestDeNorm - mean)/std

In [7]:
print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

[405, 13] [405, 1] [101, 13] [101, 1]


# Model

In [8]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

# Training

In [9]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [10]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [11]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let logits = model(XTrain[batchStart..<batchEnd])
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(XTrain[batchStart..<batchEnd])
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 632.1485, MAE: 23.148327
Epoch 2: MSE: 584.4404, MAE: 22.168392
Epoch 3: MSE: 538.1057, MAE: 21.12162
Epoch 4: MSE: 487.00983, MAE: 19.92065
Epoch 5: MSE: 430.2197, MAE: 18.5311
Epoch 6: MSE: 369.36658, MAE: 16.956282
Epoch 7: MSE: 304.5945, MAE: 15.117924
Epoch 8: MSE: 244.83635, MAE: 13.123546
Epoch 9: MSE: 191.10953, MAE: 11.197143
Epoch 10: MSE: 145.38417, MAE: 9.311882
Epoch 11: MSE: 111.12395, MAE: 7.85258
Epoch 12: MSE: 87.4424, MAE: 6.81113
Epoch 13: MSE: 70.84821, MAE: 5.9915586
Epoch 14: MSE: 58.041485, MAE: 5.3819957
Epoch 15: MSE: 48.23771, MAE: 4.868318
Epoch 16: MSE: 41.520054, MAE: 4.414794
Epoch 17: MSE: 36.932674, MAE: 4.124955
Epoch 18: MSE: 33.462967, MAE: 3.9070601
Epoch 19: MSE: 30.767656, MAE: 3.7203555
Epoch 20: MSE: 29.130291, MAE: 3.6115162
Epoch 21: MSE: 27.735817, MAE: 3.558845
Epoch 22: MSE: 26.397898, MAE: 3.4242277
Epoch 23: MSE: 25.288857, MAE: 3.310348
Epoch 24: MSE: 24.107788, MAE: 3.252024
Epoch 25: MSE: 23.275051, MAE: 3.1446037
Epoch 26

Epoch 205: MSE: 4.4935102, MAE: 1.4076074
Epoch 206: MSE: 4.3712454, MAE: 1.3961618
Epoch 207: MSE: 4.3862796, MAE: 1.4163995
Epoch 208: MSE: 4.4307966, MAE: 1.3990127
Epoch 209: MSE: 4.494864, MAE: 1.4060295
Epoch 210: MSE: 4.5203114, MAE: 1.4040179
Epoch 211: MSE: 4.341959, MAE: 1.3984491
Epoch 212: MSE: 4.4175053, MAE: 1.3900346
Epoch 213: MSE: 4.28123, MAE: 1.383579
Epoch 214: MSE: 4.280751, MAE: 1.3795216
Epoch 215: MSE: 4.099319, MAE: 1.3919181
Epoch 216: MSE: 4.301519, MAE: 1.3759516
Epoch 217: MSE: 4.225368, MAE: 1.3698826
Epoch 218: MSE: 3.9647317, MAE: 1.3670576
Epoch 219: MSE: 4.4919505, MAE: 1.3935397
Epoch 220: MSE: 4.004787, MAE: 1.3532214
Epoch 221: MSE: 4.4397483, MAE: 1.3645966
Epoch 222: MSE: 4.0781302, MAE: 1.3407657
Epoch 223: MSE: 3.8731463, MAE: 1.3399535
Epoch 224: MSE: 4.0772066, MAE: 1.3825245
Epoch 225: MSE: 4.0019073, MAE: 1.3432616
Epoch 226: MSE: 3.92673, MAE: 1.3790638
Epoch 227: MSE: 4.1864243, MAE: 1.3392556
Epoch 228: MSE: 4.2034817, MAE: 1.35068
Epoch 

Epoch 404: MSE: 2.3915553, MAE: 1.0427587
Epoch 405: MSE: 2.2404656, MAE: 1.0228733
Epoch 406: MSE: 2.3442717, MAE: 1.0850009
Epoch 407: MSE: 2.28609, MAE: 1.0832258
Epoch 408: MSE: 2.6798272, MAE: 1.0743743
Epoch 409: MSE: 2.217145, MAE: 1.0236187
Epoch 410: MSE: 2.2742932, MAE: 1.015586
Epoch 411: MSE: 2.4155526, MAE: 1.0769486
Epoch 412: MSE: 2.3099256, MAE: 1.0327656
Epoch 413: MSE: 2.30275, MAE: 1.0693271
Epoch 414: MSE: 2.289198, MAE: 1.0391394
Epoch 415: MSE: 2.5266175, MAE: 1.0879909
Epoch 416: MSE: 2.2677636, MAE: 1.003963
Epoch 417: MSE: 2.1102471, MAE: 0.98939794
Epoch 418: MSE: 2.2509553, MAE: 1.0687753
Epoch 419: MSE: 2.1873326, MAE: 1.0588976
Epoch 420: MSE: 2.6436925, MAE: 1.0745656
Epoch 421: MSE: 2.2373984, MAE: 0.9969577
Epoch 422: MSE: 2.205997, MAE: 1.0297091
Epoch 423: MSE: 2.309804, MAE: 1.0643514
Epoch 424: MSE: 2.3020053, MAE: 0.9992581
Epoch 425: MSE: 2.133378, MAE: 1.0376208
Epoch 426: MSE: 2.2426405, MAE: 1.0588949
Epoch 427: MSE: 2.1456466, MAE: 1.0613663
Ep

# Test

In [12]:
Context.local.learningPhase = .inference

let prediction = model(XTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 1.2903645, MAE: 0.089290395


In [13]:
print(XTrain[0], YTrain[0], model(XTrain[0].reshaped(to: TensorShape([1, 13]))))

print(XTest[0], YTest[0], model(XTest[0].reshaped(to: TensorShape([1, 13]))))
print(XTest[1], YTest[1], model(XTest[1].reshaped(to: TensorShape([1, 13]))))
print(XTest[17], YTest[17], model(XTest[17].reshaped(to: TensorShape([1, 13]))))

[ -0.30846816,   0.15054052,   -1.1039964,  -0.30756173,   0.05126577,   0.33352128,
  0.024851447, -0.035726987,  -0.89119065,  -0.43651816,   -1.2319369,   0.42111015,
  -0.93603975] [24.0] [[24.00957]]
[  10.127699, -0.56208307,   1.3125796, -0.30756173,   1.4050479,  -0.8863933,   1.2248203,
  -1.2581577,   2.6233904,   2.3634233,   0.9779132,  0.12650238,   1.6906141] [5.0] [[11.374844]]
[  2.8739426, -0.56208307,   1.3125796, -0.30756173,    1.108089,  -2.9993627,   1.2248203,
  -1.3716108,   2.6233904,   2.3634233,   0.9779132, -0.23774466,   1.7431473] [11.9] [[14.204795]]
[   1.541963, -0.56208307,   1.3125796, -0.30756173,   0.7150559, -0.93426037,   0.7972452,
  -1.0169381,   2.6233904,   2.3634233,   0.9779132,   -2.180478,  0.39479825] [20.8] [[30.373932]]


# Export

In [14]:
print(model.layer1.weight.shape, model.layer2.weight.shape, model.layer3.weight.shape)
print(model.layer1.bias.shape, model.layer2.bias.shape, model.layer3.bias.shape)

[13, 64] [64, 32] [32, 1]
[64] [32] [1]


In [15]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.3"]) {
    Input(name: "input", shape: [13], featureType: .Double)
    Output(name: "output", shape: [1], featureType: .Double)
    NeuralNetwork {
        InnerProduct(name: "dense1",
                     input: ["input"],
                     output: ["outDense1"],
                     weights: model.layer1.weight.transposed().flattened().scalars,
                     bias: model.layer1.bias.flattened().scalars,
                     inputChannels: 13,
                     outputChannels: 64,
                     updatable: false)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weights: model.layer2.weight.transposed().flattened().scalars,
                     bias: model.layer2.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32,
                     updatable: false)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weights: model.layer3.weight.transposed().flattened().scalars,
                     bias: model.layer3.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1,
                     updatable: false)
    }
}

In [16]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "../model/s4tf_train_model.mlmodel"))